## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-19-02-16 +0000,nypost,LA mayor candidate Austin Beutner’s daughter E...,https://nypost.com/2026/01/21/us-news/austin-b...
1,2026-01-21-18-59-30 +0000,startribune,Our reporters answered your questions about th...,https://www.startribune.com/what-do-you-want-t...
2,2026-01-21-18-58-37 +0000,nyt,Live Updates: Trump Assails Europe and Demands...,https://www.nytimes.com/live/2026/01/21/us/tru...
3,2026-01-21-18-57-30 +0000,nyt,Mediaite Starts a Newsletter to Summarize Medi...,https://www.nytimes.com/2026/01/21/business/me...
4,2026-01-21-18-56-50 +0000,cbc,"Talks over Chinese steel, aluminum hampering f...",https://www.cbc.ca/news/politics/canola-china-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2391/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,98
25,greenland,41
79,davos,23
98,new,18
261,court,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
44,2026-01-21-17-55-00 +0000,nyt,Trump Demands Control of Greenland but Rules O...,https://www.nytimes.com/2026/01/21/us/politics...,207
163,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...,205
22,2026-01-21-18-34-00 +0000,wsj,‘A Piece of Ice for World Protection’: Trump D...,https://www.wsj.com/politics/policy/a-piece-of...,194
7,2026-01-21-18-53-24 +0000,nypost,"Bessent calls Denmark ‘irrelevant,’ shrugs off...",https://nypost.com/2026/01/21/business/bessent...,193
83,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...,189


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
44,207,2026-01-21-17-55-00 +0000,nyt,Trump Demands Control of Greenland but Rules O...,https://www.nytimes.com/2026/01/21/us/politics...
66,74,2026-01-21-17-25-34 +0000,nyt,Tariffs Case Pending as Supreme Court Takes Up...,https://www.nytimes.com/2026/01/21/us/politics...
288,59,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
7,54,2026-01-21-18-53-24 +0000,nypost,"Bessent calls Denmark ‘irrelevant,’ shrugs off...",https://nypost.com/2026/01/21/business/bessent...
85,54,2026-01-21-16-50-45 +0000,nypost,New dad who just opened flight school killed i...,https://nypost.com/2026/01/21/us-news/new-dad-...
125,47,2026-01-21-15-22-38 +0000,nyt,Man Gets Life in Prison for Killing Shinzo Abe...,https://www.nytimes.com/2026/01/20/world/asia/...
60,42,2026-01-21-17-28-00 +0000,wsj,EU Lawmakers Ask for Legal Opinion on South Am...,https://www.wsj.com/economy/trade/eu-lawmakers...
242,41,2026-01-21-07-22-15 +0000,nypost,Israel’s Netanyahu agrees to join Trump’s Boar...,https://nypost.com/2026/01/21/world-news/israe...
169,41,2026-01-21-12-45-27 +0000,startribune,The latest: JD Vance to meet with ICE agents t...,https://www.startribune.com/ice-raids-minnesot...
55,38,2026-01-21-17-31-24 +0000,nypost,"One dead, 37 injured in Spanish train crash — ...",https://nypost.com/2026/01/21/world-news/anoth...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
